# Chapter 1 - Mathematical Basis

In [1]:
import sys
sys.path.append("../")
from utils import *

## Multivariate Functions

In [2]:
vals = np.linspace(-5, 5, 100)
names, funs, scenes = ["max","polynomial"], [lambda x,y: np.max([x,y], axis=0), lambda x, y: x**2 + x*y + y**2], \
    [dict(camera = dict(eye=dict(x=-1.5, y=-1.5, z=.1))), dict(camera = dict(eye=dict(x=1.5, y=-2, z=.7)))]

for name, fun, scene in zip(*[names, funs, scenes]):
    xx, yy = np.meshgrid(vals, vals)
    z = fun(xx, yy)

    fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'scatter'}, {'type': 'scene'}]])

    fig.add_traces(data = [
        go.Contour(z=z, colorscale='Electric', showscale=False),    
        go.Surface(x = vals, y=vals, z=z, opacity=.8, colorscale='Electric', contours=dict(z=dict(show=True)))],
        rows=[1,1], cols=[1,2])

    fig.update_layout(width=800, height=300, scene_aspectmode="cube", scene=scene)
    fig.write_image(f"../{name}.png")
    fig.show()
    

ValueError: Image generation requires the psutil package.

Install using pip:
    $ pip install psutil

Install using conda:
    $ conda install psutil


## Gaussian Distribution

### Standard Uni- and Bi- variate Gaussians

In [ ]:
from scipy.stats import multivariate_normal

vals = np.linspace(-5, 5, 100)
xx, yy = np.meshgrid(vals, vals)
z = multivariate_normal(mean=np.zeros(2), cov=np.eye(2)).pdf(np.c_[xx.ravel(), yy.ravel()]).reshape(len(vals), len(vals))

fig = make_subplots(rows=1, cols=2, subplot_titles=["Univariate Standard Normal", "Bivariate Standard Normal"],
                    specs=[[{'type': 'scatter'}, {'type': 'scene'}]])

fig.add_traces(data = [
    go.Scatter(x=vals, y=multivariate_normal(0, 1).pdf(vals), mode="markers", showlegend=False,
                   marker=dict(color=multivariate_normal(0, 1).pdf(vals), colorscale="Electric", size=5)),    
    go.Surface(x = vals, y=vals, z=z, opacity=.8, colorscale="Electric")
        ],
    rows=[1,1], cols=[1,2])

fig.update_layout(width=800, height=300, scene_aspectmode="cube", scene=dict(camera = dict(eye=dict(x=1.6, y=1.6, z=.3))))
fig.write_image("../normal_distributions.png")
fig.show()



### Linear Transformations Influencing Sample Covariance Matrix
We begin with sampling 750 data-points from a bivariate Gaussian

In [ ]:
import pandas as pd
np.random.seed(0)

X = np.random.multivariate_normal([0,0], np.eye(2), 750)
pd.DataFrame(np.cov(X.T,ddof=1), columns=["Covariate 1", "Covariate 2"], index=["Covariate 1", "Covariate 2"])

In [ ]:
fig = go.Figure(go.Scatter(x=X[:,0], y=X[:,1], mode="markers", marker=dict(color="black")),
                layout=dict(xaxis=dict(range=[-6,6], autorange=False),
                            yaxis=dict(range=[-6,6], autorange=False),
                            width=400, height=400))
fig.write_image("../cov.png")
fig.show()

Then, we scale the data using a diagonal matrix. Scaling is different for the different covariates.

In [ ]:
S = np.array([[1.2,0],[0,2]])
X = X @ S
pd.DataFrame(np.cov(X.T,ddof=1), columns=["Covariate 1", "Covariate 2"], index=["Covariate 1", "Covariate 2"])

In [ ]:
fig = go.Figure(go.Scatter(x=X[:,0], y=X[:,1], mode="markers", marker=dict(color="black")),
                layout=dict(xaxis=dict(range=[-6,6], autorange=False),
                            yaxis=dict(range=[-6,6], autorange=False),
                            width=400, height=400))
fig.write_image("../scaled_cov.png")
fig.show()

Lastly, we rotate the data by an angle of $\theta=70$ degrees.

In [ ]:
theta = 70
R = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
X = X @ R
pd.DataFrame(np.cov(X.T,ddof=1), columns=["Covariate 1", "Covariate 2"], index=["Covariate 1", "Covariate 2"])

In [ ]:
fig = go.Figure(go.Scatter(x=X[:,0], y=X[:,1], mode="markers", marker=dict(color="black")),
                layout=dict(xaxis=dict(range=[-6,6], autorange=False),
                            yaxis=dict(range=[-6,6], autorange=False),
                            width=400, height=400))
fig.write_image("../correlated_cov.png")
fig.show()